# Pace Correlations

Does pace in the clutch correlate with net rating in the clutch for teams? A simple linear regression.

In [ ]:
import pandas as pd

Data is from nba.com, in nba_selected_stats.csv

In [ ]:
df = pd.read_csv("../../data/nba_selected_stats.csv")
df

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Prepare data for regression
X = df["PACE"].values.reshape(-1, 1)  # Feature
y = df["NetRtg"].values  # Response

# Linear regression
reg = LinearRegression()
reg.fit(X, y)

# Prediction
y_pred = reg.predict(X)


# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(X, y, color="blue", label="Data Points", edgecolors="k", s=100)
plt.plot(X, y_pred, color="red", label="Regression Line", linewidth=2)

# Add team names as labels next to data points
for i in range(len(df)):
    plt.text(
        X[i],
        y[i],
        df["Team"][i].split(" ")[-1],
        fontsize=10,
        ha="right",
        va="bottom",
        color="darkblue",
    )

plt.title("Linear Regression: PACE vs NetRtg", fontsize=16)
plt.xlabel("PACE", fontsize=14)
plt.ylabel("NetRtg", fontsize=14)
plt.grid(True)
plt.legend()
plt.tight_layout()

# Show plot
plt.show()

# Display the linear regression equation
print(
    f"Linear Regression Equation: NetRtg = {reg.intercept_:.2f} + {reg.coef_[0]:.2f} * PACE"
)

# PCA on Clutch Stats

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Create the dataframe
data = {
    "TeamId": [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19,
        20,
        21,
        22,
        23,
        24,
        25,
        26,
        27,
        28,
        29,
        30,
    ],
    "TeamName": [
        "Washington Wizards",
        "Atlanta Hawks",
        "Portland Trail Blazers",
        "Milwaukee Bucks",
        "Charlotte Hornets",
        "Golden State Warriors",
        "Orlando Magic",
        "Toronto Raptors",
        "Chicago Bulls",
        "Detroit Pistons",
        "Philadelphia 76ers",
        "LA Clippers",
        "Memphis Grizzlies",
        "Phoenix Suns",
        "Sacramento Kings",
        "Indiana Pacers",
        "New Orleans Pelicans",
        "Denver Nuggets",
        "Boston Celtics",
        "Los Angeles Lakers",
        "San Antonio Spurs",
        "Minnesota Timberwolves",
        "Houston Rockets",
        "New York Knicks",
        "Dallas Mavericks",
        "Cleveland Cavaliers",
        "Utah Jazz",
        "Brooklyn Nets",
        "Oklahoma City Thunder",
        "Miami Heat",
    ],
    "GP": [
        20,
        20,
        23,
        26,
        25,
        24,
        20,
        19,
        21,
        19,
        20,
        24,
        27,
        29,
        16,
        25,
        19,
        23,
        19,
        16,
        21,
        20,
        25,
        20,
        20,
        20,
        15,
        28,
        30,
        18,
    ],
    "W": [
        15,
        12,
        14,
        12,
        14,
        14,
        11,
        7,
        12,
        11,
        9,
        11,
        17,
        15,
        9,
        16,
        10,
        10,
        9,
        9,
        11,
        9,
        14,
        9,
        9,
        6,
        5,
        9,
        13,
        4,
    ],
    "L": [
        5,
        8,
        9,
        14,
        11,
        10,
        9,
        12,
        9,
        8,
        11,
        13,
        10,
        14,
        7,
        9,
        9,
        13,
        10,
        7,
        10,
        11,
        11,
        11,
        11,
        14,
        10,
        19,
        17,
        14,
    ],
    "WIN%": [
        75.0,
        60.0,
        60.9,
        46.2,
        56.0,
        58.3,
        55.0,
        36.8,
        57.1,
        57.9,
        45.0,
        45.8,
        63.0,
        51.7,
        56.3,
        64.0,
        52.6,
        43.5,
        47.4,
        56.3,
        52.4,
        45.0,
        56.0,
        45.0,
        45.0,
        30.0,
        33.3,
        32.1,
        43.3,
        22.2,
    ],
    "Min": [
        2.9,
        3.4,
        4.5,
        4.1,
        4.1,
        3.9,
        3.6,
        4.3,
        3.1,
        3.6,
        3.1,
        3.4,
        4.1,
        2.9,
        3.0,
        3.2,
        4.4,
        4.9,
        3.0,
        4.1,
        4.1,
        3.1,
        3.6,
        4.2,
        4.2,
        3.9,
        2.4,
        3.9,
        3.4,
        4.1,
    ],
    "PTS": [
        8.8,
        8.3,
        11.8,
        10.5,
        10.9,
        10.3,
        8.9,
        9.6,
        7.8,
        8.5,
        7.5,
        8.0,
        9.3,
        7.2,
        7.2,
        7.4,
        9.5,
        9.5,
        7.6,
        9.8,
        9.7,
        6.7,
        8.0,
        8.5,
        9.3,
        8.5,
        4.7,
        8.4,
        6.7,
        8.3,
    ],
    "FGM": [
        2.8,
        2.4,
        3.9,
        3.6,
        3.3,
        3.5,
        2.7,
        3.4,
        2.6,
        2.6,
        2.5,
        2.5,
        2.9,
        2.2,
        2.2,
        1.9,
        3.1,
        3.3,
        2.5,
        3.7,
        2.8,
        2.1,
        2.7,
        2.6,
        3.1,
        3.0,
        1.5,
        2.5,
        2.2,
        2.4,
    ],
    "FGA": [
        5.0,
        5.4,
        8.0,
        7.5,
        7.2,
        6.8,
        5.3,
        7.2,
        5.0,
        5.8,
        5.5,
        6.3,
        7.4,
        5.2,
        4.8,
        4.8,
        7.3,
        8.3,
        5.2,
        6.9,
        6.6,
        5.1,
        6.1,
        6.6,
        7.3,
        6.6,
        3.9,
        6.9,
        5.5,
        6.5,
    ],
    "FG%": [
        55.0,
        44.9,
        48.6,
        47.4,
        45.3,
        51.5,
        50.0,
        46.7,
        51.4,
        45.5,
        45.5,
        40.4,
        39.2,
        43.0,
        46.1,
        39.7,
        41.7,
        39.6,
        49.0,
        53.2,
        41.7,
        41.2,
        44.1,
        38.9,
        42.5,
        45.0,
        39.0,
        36.8,
        39.2,
        36.8,
    ],
    "3PM": [
        0.9,
        0.5,
        1.6,
        0.9,
        1.1,
        0.9,
        0.6,
        0.8,
        1.0,
        1.0,
        0.4,
        0.9,
        0.9,
        1.0,
        1.9,
        0.9,
        0.7,
        0.9,
        0.7,
        0.9,
        1.0,
        1.0,
        0.7,
        1.0,
        1.2,
        0.6,
        0.6,
        0.9,
        0.7,
        0.7,
    ],
    "3PA": [
        2.0,
        2.4,
        4.4,
        2.6,
        3.0,
        2.1,
        1.9,
        2.3,
        2.2,
        2.8,
        1.8,
        3.4,
        3.2,
        2.3,
        2.6,
        2.1,
        3.0,
        3.6,
        1.8,
        2.3,
        2.5,
        2.5,
        2.3,
        3.2,
        3.4,
        2.3,
        2.7,
        2.7,
        3.2,
        2.7,
    ],
    "3P%": [
        45.0,
        21.3,
        36.3,
        33.8,
        37.8,
        43.1,
        31.6,
        34.9,
        44.7,
        35.8,
        20.0,
        27.2,
        26.7,
        40.7,
        35.1,
        29.8,
        34.0,
        28.9,
        37.1,
        27.8,
        34.0,
        32.4,
        34.0,
        31.7,
        33.8,
        30.0,
        23.8,
        24.9,
        23.8,
        27.3,
    ],
    "FTM": [
        2.4,
        3.0,
        2.4,
        2.5,
        3.2,
        2.4,
        3.0,
        2.1,
        1.6,
        2.3,
        2.1,
        2.0,
        2.7,
        2.4,
        2.3,
        2.9,
        2.4,
        2.3,
        2.3,
        2.1,
        3.3,
        2.0,
        2.4,
        2.4,
        2.0,
        2.0,
        2.0,
        2.8,
        3.0,
        3.8,
    ],
    "FTA": [
        3.2,
        3.9,
        3.3,
        3.3,
        3.8,
        3.3,
        3.4,
        2.7,
        2.5,
        3.3,
        3.4,
        3.1,
        3.7,
        3.5,
        3.5,
        3.8,
        3.6,
        3.7,
        3.3,
        3.5,
        3.5,
        3.1,
        3.5,
        3.6,
        3.6,
        3.6,
        2.7,
        3.2,
        3.4,
        3.8,
    ],
    "FT%": [
        74.9,
        77.1,
        73.0,
        75.8,
        84.2,
        72.5,
        88.2,
        76.5,
        63.0,
        69.9,
        61.8,
        64.7,
        72.8,
        68.6,
        66.4,
        75.9,
        66.6,
        62.8,
        69.8,
        60.0,
        94.9,
        66.2,
        67.1,
        67.4,
        56.7,
        56.9,
        74.3,
        87.5,
        87.0,
        83.3,
    ],
}

df = pd.DataFrame(data)

# Step 2: Normalize the data (Standard Scaling)
scaler = StandardScaler()
features = df.drop(columns=["TeamId", "TeamName"])
scaled_data = scaler.fit_transform(features)

# Step 3: Apply PCA and fit with n_components=5
pca = PCA(n_components=5)
principal_components = pca.fit_transform(scaled_data)

# Create a new DataFrame with principal components
pca_df = pd.DataFrame(
    data=principal_components, columns=[f"PC{i + 1}" for i in range(5)]
)

# Display the explained variance ratio to understand how much variance each principal component explains
print("Explained variance ratio of each principal component:")
print(pca.explained_variance_ratio_)

# Optional: Visualize explained variance ratio (Bar chart)
plt.figure(figsize=(8, 6))
sns.barplot(x=[f"PC{i + 1}" for i in range(5)], y=pca.explained_variance_ratio_)
plt.title("Explained Variance Ratio by Principal Component")
plt.ylabel("Explained Variance Ratio")
plt.show()

# Step 4: Display PCA components (the weights of each feature)
print("\nPrincipal Components (weights of each feature in the PC space):")
for i, component in enumerate(pca.components_):
    print(f"\nPrincipal Component {i + 1}:")
    for j, weight in enumerate(component):
        print(f"{features.columns[j]}: {weight:.4f}")

# Optional: Combine PCA components with the team names
final_df = pd.concat([df[["TeamId", "TeamName"]], pca_df], axis=1)
print(final_df.head())